In [6]:
from datasets import load_dataset

In [7]:
data = load_dataset('financial_phrasebank', 'sentences_50agree')

Generating train split: 100%|██████████| 4846/4846 [00:00<00:00, 65929.05 examples/s]


In [8]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis",model=model, tokenizer= tokenizer)

Device set to use cuda:0


In [9]:
def tempo(val):
    print(x_train[val])
    dict = {
        0: 'negative',
        1: 'neutral',
        2: 'positive'
    }
    result = dict[y_train[val]]
    print(result)
    print(classifier.predict(x_train[val]))

In [10]:
import requests

url = ('https://newsapi.org/v2/top-headlines?sources=google-news-in&category=business&apiKey=cef60813d9da4655ae7fe2ef2446060a')

url3 = ('https://newsapi.org/v2/top-headlines?country=in&category=business&apiKey=cef60813d9da4655ae7fe2ef2446060a')

def GetNewsOnEverything(query):
    url2 = (f'https://newsapi.org/v2/everything?q={query}&apiKey=cef60813d9da4655ae7fe2ef2446060a')
    response = requests.get(url2)
    dict = {}
    # print(response.json())
    dict = response.json()
    return dict

newsDict = GetNewsOnEverything("Infosys")

In [11]:
def get_joined_string(newsDict):
    temp_string = []
    for i in range(len(newsDict['articles'])):
        temp = newsDict['articles'][i]
        title = temp['title']
        description = temp['description']
        content = temp['content']
        try:
            final_string = title+description+content
        except ValueError:
            if type(title) == None:
                title = ""
            if type(description) == None:
                description = ""
            if type(content) == None:
                content = ""
            final_string = title+description+content
        except Exception as e:
            print("An error occurred:", e)
        temp_string.append(final_string)
    return temp_string

In [12]:
def tempo(score_list):
    dict = {
        'negative': -1,
        'neutral': 0,
        'positive': 1
    }
    score = []
    for i in score_list:
        temp = i['label']
        temp = dict[temp]
        score.append(temp)
    total_score = sum(score) 
    return total_score

In [13]:
tempo(score_list)

NameError: name 'score_list' is not defined